# 산문시
* 정의

    * 행과 연의 구분이 없는 시
    
    * 구체적으로 정의하자면, 짧고 압축된 형태로 행과 연을 파괴한 줄글이며 상상, 리듬, 표현의 밀도 등 시적 장치를 지닌 하나의 예술 형태이다.



* 산문시 vs 운문시 vs 산문

<img src="산문시.png" width="700" height="700">

* 이 프로젝트에서 **산문시**란, **한 단락**으로 되어 있으며 **시를 미적으로 만드는 형식(운율, 함축, 모호성, 이미지)**을 갖추고 있는 문학의 형태이다.

* 따라서 산문시의 형태를 취하고 있더라도 미적인 요소가 없으면 데이터에서 제외시킨다.

* 도움되는 글
    * [산문시에 관하여(평론)](https://teen.munjang.or.kr/archives/14886)


# Dataset

* Transfer Learning 방식을 사용할 것이므로 **상대적으로 양이 적어질지라도 데이터의 품질에 집중한다!**

* 어디서 크롤링할 것인가?
    * 문학광장 글틴: https://teen.munjang.or.kr/archives/category/write/poetry
    * 포스타입 <산문시> 태그: https://www.postype.com/search?keyword=%EC%82%B0%EB%AC%B8%EC%8B%9C&options_tags=1

# 크롤링

### 1. 포스타입

In [2]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import csv
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# selenium 로그 숨기기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
# 크롬 브라우저 화면 띄우지 않기
options.add_argument("headless")

In [118]:
def make_url_lists(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    posts = soup.find_all("div", attrs = {"class":"post-media-body"})
    for post in posts:
        post_url = post.find("a", attrs = {"class": "post-title-wrap ellipsis-2"})['href']
        url_list.append(post_url)
    return url_list

In [176]:
# 본문 긁어오기
driver = webdriver.Chrome(options = options)

def scraping(url_list, pList):
    time.sleep(1)
    try:
        for url in url_list:
            driver.get(url)
            title = driver.find_element(By.XPATH, "//*[@id='post']/div/header/div[1]/h1")
            contents = driver.find_element(By.XPATH, "//*[@id='post-content']")
            pList.append([title.text, contents.text])
    except NoSuchElementException:
            print("No element exists")

In [177]:
def saveToFile(filename, pList):
    with open(filename, 'w', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(pList)

In [179]:
pList = []
for i in range(1, 37):
    url_list = []
    url = "https://www.postype.com/search?page="+str(i)+"&keyword=%EC%82%B0%EB%AC%B8%EC%8B%9C&options_tags=1" # 1 page 선택
    url_list = make_url_lists(url) # 1 page에 있는 모든 포스트들의 url을 list로 받음
    scraping(url_list, pList)
saveToFile("ptype.csv", pList)

### 2. 글틴 - 7725건

In [1]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import csv
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# selenium 로그 숨기기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
# 크롬 브라우저 화면 띄우지 않기
options.add_argument("headless")

In [2]:
def make_url_lists(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    posts = soup.find_all("div", attrs = {"class": "entry-content"}) 
    for post in posts:
        content = post.find("div", attrs = {"class" : "post_title"})
        title = content.text
        if '공지' in title:
            continue
        if '장원' in title:
            continue
        url = content.find("a")['href']
        url_list.append(url)
    return url_list

In [3]:
# 본문 긁어오기
def scraping(url_list, pList):
    time.sleep(1)
    try:
        for url in url_list:
            driver.get(url)
            title = driver.find_element(By.CLASS_NAME, "entry-title")
            contents = driver.find_element(By.CLASS_NAME, "entry-content")
            pList.append([title.text, contents.text])
    except NoSuchElementException:
            print("No element exists")

In [4]:
def saveToFile(filename, pList):
    with open(filename, 'w', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(pList)

In [5]:
driver = webdriver.Chrome(options = options)

In [6]:
pList = []
start = time.time()
for i in range(1, 400):
    url_list = []
    url = "https://teen.munjang.or.kr/archives/category/write/poetry/page/" + str(i) # 1 page 선택
    url_list = make_url_lists(url) # 1 page에 있는 모든 포스트들의 url을 list로 받음
    scraping(url_list, pList)
saveToFile("teenpage501-800.csv", pList) 
end = time.time()
print("약 {}분 소요".format((end - start)//60))

약 55.0분 소요
